In [1]:
# Importing necessary libraries to extract the data properly from Postgres using PySpark
import pandas as pd
import numpy as np
from API_Key import username, password
import requests
import time
import psycopg2 as pg2
import json
from pandas.io.json import json_normalize
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as psf

In [2]:
spark = SparkSession \
    .builder \
    .appName("TFT Analysis") \
    .config("spark.jars", "/Users/hiowatah/downloads/postgresql-42.2.18.jar") \
    .master("local[5]") \
    .getOrCreate()

In [3]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/TFT") \
    .option("dbtable", "matches") \
    .option("user", username) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [4]:
df.schema

StructType(List(StructField(match,StringType,true),StructField(game_datetime,LongType,true),StructField(game_length,DoubleType,true),StructField(game_version,StringType,true),StructField(gold_left,IntegerType,true),StructField(last_round,IntegerType,true),StructField(level,IntegerType,true),StructField(placement,IntegerType,true),StructField(puuid,StringType,true),StructField(time_eliminated,DoubleType,true),StructField(total_damage,IntegerType,true),StructField(traits,StringType,true),StructField(units,StringType,true)))

In [5]:
def jsonColParser(df, *cols, sanitize=True):
    res = df
    for i in cols:
        if sanitize:
            res = (
                res.withColumn(
                    i,
                    psf.concat(psf.lit('{"data": '), i, psf.lit('}'))
                )
            )
        schema = spark.read.json(res.rdd.map(lambda x: x[i])).schema
        res = res.withColumn(i, psf.from_json(psf.col(i), schema))
        
        if sanitize:
            res = res.withColumn(i, psf.col(i).data)
            
    return res

## Creating second dataframe of exploded columns because each player is going to take up multiple rows of data for each unit and trait they had for the game. Analysis will be easier by filtering the first dataframe and getting the match info from the second dataframe

In [6]:
df2 = jsonColParser(df, "units", "traits")
df2 = df.where("game_version LIKE '%Version 11%'")

In [7]:
df2 = df.withColumn("unit_names", psf.explode(df.units.character_id))\
        .withColumn("units_tier", psf.explode(df.units.tier))\
        .withColumn("chosen", psf.explode(df.units.chosen))\
        .withColumn("items", psf.explode(df.units.items))\
        .withColumn("trait_name", psf.explode(df.traits['name']))\
        .withColumn("num_units_trait", psf.explode(df.traits.num_units))\
        .withColumn("trait_tier", psf.explode(df.traits.tier_current))\
        .withColumn("style", psf.explode(df.traits.style))\
        .withColumn("trait_tier_total", psf.explode(df.traits.tier_total))\
        .drop("units", "traits")

In [8]:
df.schema

StructType(List(StructField(match,StringType,true),StructField(game_datetime,LongType,true),StructField(game_length,DoubleType,true),StructField(game_version,StringType,true),StructField(gold_left,IntegerType,true),StructField(last_round,IntegerType,true),StructField(level,IntegerType,true),StructField(placement,IntegerType,true),StructField(puuid,StringType,true),StructField(time_eliminated,DoubleType,true),StructField(total_damage,IntegerType,true),StructField(unit_names,StringType,true),StructField(units_tier,LongType,true),StructField(chosen,StringType,true),StructField(items,ArrayType(LongType,true),true),StructField(trait_name,StringType,true),StructField(num_units_trait,LongType,true),StructField(trait_tier,LongType,true),StructField(style,LongType,true),StructField(trait_tier_total,LongType,true)))